### Load minecraft-data (First Dataset)

In [50]:
import json

artic_path = 'data\\ArticData\\'

# minecraft data
minecraft_data_path = 'data\\minecraft-data-3.63.0\\data\pc\\1.20\\'
files = ['biomes', 'blockLoot', 'blocks', 'effects', 'enchantments', 'entities', 'entityLoot', 'foods', 'items', 'materials', 'particles', 'recipes']
minecraft_data = {}

for file in files:
    with open(minecraft_data_path + file + '.json') as f:
        minecraft_data[file] = json.load(f)

### Load Artic (Second Dataset) and prepare id-mappings

In [51]:
# items
artic_items = {}
with open(artic_path + '1_20_items.json') as f:
    artic_items = json.load(f)
md_items = {item_data['id']: item_data for item_data in minecraft_data['items']}

# blocks
md_blocks = {block_data['id']: block_data for block_data in minecraft_data['blocks']}
artic_blocks = {}
with open(artic_path + '1_20_blocks.json') as f:
    artic_blocks = json.load(f)

# block loot
md_block_loot = {block_data['block']: block_data['drops'] for block_data in minecraft_data['blockLoot']}

# effects
artic_effects = {}
with open(artic_path + '1_20_potion_effects.json') as f:
    artic_effects = json.load(f)
md_effects = {effect_data['id']: effect_data for effect_data in minecraft_data['effects']}

# enchantments
artic_enchantments = {}
with open(artic_path + '1_20_enchantments.json') as f:
    artic_enchantments = json.load(f)
md_enchantments = {enchantment_data['id']: enchantment_data for enchantment_data in minecraft_data['enchantments']}

# biomes
artic_biomes = {}
with open(artic_path + '1_20_biomes.json') as f:
    artic_biomes = json.load(f)
md_biomes = {f"minecraft:{biome_data['name']}": biome_data for biome_data in minecraft_data['biomes']}

# entities
artic_entities = {}
with open(artic_path + '1_20_entities.json') as f:
    artic_entities = json.load(f)
md_entities = {f"minecraft:{entity_data['name']}": entity_data for entity_data in minecraft_data['entities']}

# dimensions
artic_dimensions = {}
with open(artic_path + '1_20_dimension_types.json') as f:
    artic_dimensions = json.load(f)

# entity loot
md_entity_loot = {entity_data['entity']: entity_data['drops'] for entity_data in minecraft_data['entityLoot']}

# chest loot
artic_chest_loot = {}
with open(artic_path + '1_20_loot_tables\\1_20_chest_loot_tables.json') as f:
    artic_chest_loot = json.load(f)

# recipes
artic_recipes = {}
with open(artic_path + '1_20_recipes.json') as f:
    artic_recipes = json.load(f)

# name from minecraft_id
def name_from_minecraft_id(minecraft_id):
    return minecraft_id.split(':')[1]

### Items

In [52]:
items = {}

for minecraft_id, artic in artic_items.items():
    item_id = artic['id']
    item_name = name_from_minecraft_id(minecraft_id)
    md = md_items[item_id]
    item = {}

    item['name'] = item_name
    item['id'] = item_id
    item['minecraft_id'] = minecraft_id
    item['name'] = md['name']
    item['display_name'] = md['displayName']
    item['stack_size'] = md['stackSize']
    item['rarity'] = artic['rarity']
    item['fire_resistant'] = artic['fireResistant']
    
    if 'foodProperties' in artic:
        foodProperties = artic['foodProperties']
        effects = [{'effect': name_from_minecraft_id(effect['id']),
                    'duration': effect['duration'], 
                    'amplifier': effect['amplifier'], 
                    'chance': effect['chance']} for effect in foodProperties['effects']]
        
        # add food properties
        item['foodProperties'] = {
            'nutrition': foodProperties['nutrition'],
            'saturation_modifier': foodProperties['saturationModifier'],
            'effects': effects,
        }

    items[item_name] = item

### Chest Loot

In [53]:
for chest_location in artic_chest_loot:
    for pool in artic_chest_loot[chest_location]['pools']:
        for entry in pool['entries']:
            if 'name' in entry:
                item = items[name_from_minecraft_id(entry['name'])]
                if 'chest_locations' not in item:
                    item['chest_locations'] = [chest_location]
                elif chest_location not in item['chest_locations']:
                    item['chest_locations'].append(chest_location)

### Recipes

In [54]:
recipes = {}
from collections import Counter

def get_shaped_recipe_ingredients(recipe):
    ingredients = {}
    keys = recipe['key']
    pattern = recipe['pattern']
    joined_pattern = ''.join(pattern)
    for key, ingredient in keys.items():
        if 'item' in ingredient: # TODO this can also be a tag for item groups
            ingredients[name_from_minecraft_id(ingredient['item'])] = joined_pattern.count(key)
    return ingredients

def get_shapeless_recipe_ingredients(recipe):
    ingredients = []
    for ingredient in recipe['ingredients']:
        if 'item' in ingredient: # TODO this can also be a tag for item groups
            ingredients.append(name_from_minecraft_id(ingredient['item']))
    ingredients = Counter(ingredients)
    return ingredients

def link_to_ingredients(recipe):
    for item_id in recipe['ingredients']:
        item = items[item_id]
        if 'used_in_recipes' not in item:
            item['used_for'] = [recipe['minecraft_id']]
        else:
            item['used_for'].append(recipe['minecraft_id'])

def link_to_result(recipe):
    item = items[recipe['item']]
    if 'recipes' not in item:
        item['crafted_with'] = [recipe['minecraft_id']]
    else:
        item['crafted_with'].append(recipe['minecraft_id'])

for minecraft_id, artic in artic_recipes.items():
    if 'result' not in artic:
        continue
    recipe = {}
    recipe_name = name_from_minecraft_id(minecraft_id)
    recipe_type = artic['type']
    recipe['name'] = recipe_name
    recipe['minecraft_id'] = minecraft_id
    recipe['type'] = recipe_type

    result = artic['result']
    if type(result) == str:
        recipe['item'] = name_from_minecraft_id(result)
        recipe['count'] = artic['count'] if 'count' in artic else 1
    else:
        recipe['item'] = name_from_minecraft_id(result['item'])
        recipe['count'] = result['count'] if 'count' in result else 1

    if recipe_type == 'minecraft:crafting_shaped':
        recipe['ingredients'] = get_shaped_recipe_ingredients(artic)
    elif recipe_type == 'minecraft:crafting_shapeless':
        recipe['ingredients'] = get_shapeless_recipe_ingredients(artic)
    # TODO is this everything?
    
    if 'ingredients' in recipe:
        link_to_ingredients(recipe)

    link_to_result(recipe)

    recipes[recipe_name] = recipe

### Blocks / Block Loot

In [55]:
blocks = {}

# prepare block drops
def get_block_drops(drops):
    block_drops = []
    for drop in drops:
        silk_touch = 'optional'
        if 'silkTouch' in drop:
            silk_touch = 'required'
        elif 'noSilkTouch' in drop:
            silk_touch = 'forbidden'
        block_drops.append({
            'item': drop['item'],
            'min': drop['stackSizeRange'][0],
            'max': drop['stackSizeRange'][1],
            'chance': drop['dropChance'],
            'silk_touch': silk_touch
        })
    return block_drops

for minecraft_id, artic in artic_blocks.items():
    block_id = artic['id']
    md = md_blocks[block_id]
    block_name = md['name']

    block = {}
    block['name'] = block_name
    block['block_id'] = block_id
    block['minecraft_id'] = minecraft_id

    # link to corresponding item
    if 'correspondingItem' in artic:
        corresponding_item_name = name_from_minecraft_id(artic['correspondingItem'])
        block['corresponding_item'] = corresponding_item_name
        items[corresponding_item_name]['block_id'] = block_id

    block['display_name'] = md['displayName']
    block['hardness'] = md['hardness']
    block['explosion_resistance'] = artic['explosionResistance']
    block['diggable'] = md['diggable']
    block['gravity'] = artic['gravity']
    block['transparent'] = md['transparent']
    block['emit_light'] = md['emitLight']
    block['filter_light'] = md['filterLight']
    block['drops'] = []

    # some blocks have a base variant (wallSign -> sign, etc.)
    # drops are mapped to the base block only
    base_block_name = items[corresponding_item_name]['name'] # find base block
    if base_block_name in md_block_loot:
        md_drops = md_block_loot[base_block_name]
        block['drops'] = get_block_drops(md_drops)         # add base block drops

    # TODO does this mapping work for every blockLoot?

    blocks[block_name] = block

### Effects

In [56]:
effects = {}

for minecraft_id, artic in artic_effects.items():
    effect_id = artic['id']
    effect_name = name_from_minecraft_id(minecraft_id)
    md = md_effects[effect_id]

    effect = {}
    effect['name'] = effect_name
    effect['minecraft_id'] = minecraft_id
    effect['display_name'] = md['displayName']
    effect['isGood'] = md['type'] == 'good'
    effect['instantaneous'] = artic['instantaneous']

    effects[effect_name] = effect

### Enchantments

In [57]:
enchantments = {}

for minecraft_id, artic in artic_enchantments.items():
    enchantment_id = artic['id']
    enchantment_name = name_from_minecraft_id(minecraft_id)
    md = md_enchantments[enchantment_id]

    enchantment = {}
    enchantment['name'] = enchantment_name
    enchantment['id'] = enchantment_id
    enchantment['minecraft_id'] = minecraft_id
    enchantment['display_name'] = md['displayName']
    enchantment['min_level'] = artic['minLevel']
    enchantment['max_level'] = artic['maxLevel']
    enchantment['rarity'] = artic['rarity']
    enchantment['category'] = artic['category']
    enchantment['weight'] = md['weight']
    enchantment['curse'] = artic['curse']
    enchantment['discoverable'] = artic['discoverable']
    enchantment['tradeable'] = artic['tradeable']
    enchantment['treasure_only'] = artic['treasureOnly']
    enchantment['excludes'] = [name_from_minecraft_id(minecraft_id) for minecraft_id in artic['incompatibleEnchantments']]

    enchantments[enchantment_name] = enchantment

### Entities / Entity Loot

In [58]:
entities = {}

def get_entity_drops(drops):
    entity_drops = {}
    for drop in drops:
        requires_player_kill = False
        if 'requiresPlayerKill' in drop:
            requires_player_kill = True

        entity_drops[drop['item']] = {
            'min': drop['stackSizeRange'][0],
            'max': drop['stackSizeRange'][1],
            'chance': drop['dropChance'],
            'requires_player_kill': requires_player_kill
        }

    return entity_drops

for minecraft_id, artic in artic_entities.items():
    md = md_entities[minecraft_id]
    entity_name = name_from_minecraft_id(minecraft_id)

    entity = {}
    entity['name'] = entity_name
    entity['minecraft_id'] = minecraft_id
    entity['display_name'] = md['displayName']
    entity['type'] = md['type']
    entity['packet_type'] = artic['packetType']
    entity['category'] = md['category']
    entity['height'] = artic['height']
    entity['width'] = artic['width']
    entity['fire_immune'] = artic['fireImmune']
    entity['drops'] = {}
    if entity_name in md_entity_loot:
        entity['drops'] = get_entity_drops(md_entity_loot[entity_name])
        for item_id in entity['drops']:
            item = items[item_id]
            if 'dropped_by_entities' not in item:
                item['dropped_by_entities'] = [minecraft_id]
            else:
                item['dropped_by_entities'].append(minecraft_id)

    entities[entity_name] = entity

### Dimensions

In [59]:
dimensions = {}

for minecraft_id, artic in artic_dimensions.items():
    dimension_name = name_from_minecraft_id(minecraft_id)

    dimension = {}
    dimension['name'] = dimension_name
    dimension['minecraft_id'] = minecraft_id
    dimension['coordinate_scale'] = artic['coordinate_scale']
    dimension['has_ceiling'] = artic['has_ceiling']
    dimension['has_skylight'] = artic['has_skylight']
    dimension['ultrawarm'] = artic['ultrawarm']
    dimension['natural'] = artic['natural']
    dimension['piglin_safe'] = artic['piglin_safe']
    dimension['respawn_anchor_works'] = artic['respawn_anchor_works']
    dimension['bed_works'] = artic['bed_works']
    dimension['has_raids'] = artic['has_raids']
    dimension['logical_height'] = artic['logical_height']
    dimension['min_y'] = artic['min_y']

    dimensions[dimension_name] = dimension

### Biomes

In [60]:
biomes = {}

for minecraft_id, artic in artic_biomes.items():
    md = md_biomes[minecraft_id]
    block_name = name_from_minecraft_id(minecraft_id)

    biome = {}
    biome['name'] = block_name
    biome['minecraft_id'] = minecraft_id
    biome['display_name'] = md['displayName']
    biome['downfall'] = artic['downfall']
    biome['has_precipitation'] = artic['has_precipitation']
    biome['temperature'] = artic['temperature']
    biome['category'] = md['category']
    biome['dimension'] = md['dimension']

    biomes[block_name] = biome

### Write to JSON

In [61]:
with open('preparedData/blocks.json', 'w') as f:
    json.dump(blocks, f, indent=4)

with open('preparedData/items.json', 'w') as f:
    json.dump(items, f, indent=4)

with open('preparedData/effects.json', 'w') as f:
    json.dump(effects, f, indent=4)

with open('preparedData/enchantments.json', 'w') as f:
    json.dump(enchantments, f, indent=4)

with open('preparedData/biomes.json', 'w') as f:
    json.dump(biomes, f, indent=4)

with open('preparedData/entities.json', 'w') as f:
    json.dump(entities, f, indent=4)

with open('preparedData/dimensions.json', 'w') as f:
    json.dump(dimensions, f, indent=4)

with open('preparedData/recipes.json', 'w') as f:
    json.dump(recipes, f, indent=4)